# SimulationArchives with REBOUNDx

We can reproduce `REBOUNDx` simulations with extra effects bit by bit with the `SimulationArchive` (see Rein & Tamayo 2017 for details), and make sure to read [SimulationArchive.ipynb](https://github.com/hannorein/rebound/blob/master/ipython_examples/SimulationArchive.ipynb) and [SimulationArchiveRestart.ipynb](https://github.com/hannorein/rebound/blob/master/ipython_examples/SimulationArchiveRestart.ipynb) first.

However, this will only work under some conditions. In particular, for bit-by-bit reproducibility, one requires that:
- All `REBOUNDx` effects in the simulation are machine independent (can't have, e.g., trig functions, pow or exp in implementation)
- The effect and particle parameters cannot change throughout the simulation
- Effects must remain on for the entire integration

in addition to the base conditions for a pure REBOUND simulation (see Rein & Tamayo 2017):
- Must use the `IAS15` or `WHFast` integrator
- Number of particles can't change throughout the simulation

To use the `SimulationArchive` with REBOUNDx, we need to save a REBOUNDx binary [SavingAndLoadingSimulations.ipynb](https://github.com/dtamayo/reboundx/blob/master/ipython_examples/SavingAndLoadingSimulations.ipynb). Since we can't change effects or particle parameters, it doesn't matter at what point we save this binary (we will just need it to load the `SimulationArchive`). Here we do a `WHFAST` integration with the symplectic `gr_potential` implementation for general relativity corrections. We set up the simulation like we usually would:

In [1]:
import rebound
import reboundx
from reboundx import constants

sim = rebound.Simulation()
sim.add(m=1.)
sim.add(m=1e-3, a=1., e=0.2)
sim.add(m=1e-3, a=1.9)
sim.move_to_com()
sim.dt = sim.particles[1].P*0.05  # timestep is 5% of orbital period
sim.integrator = "whfast"
rebx = reboundx.Extras(sim)
gr = rebx.add("gr_potential")
gr.params["c"] = constants.C
rebx.save("rebxarchive.bin")

We now set up the `SimulationArchive` and integrate like we normally would ([SimulationArchive.ipynb](https://github.com/hannorein/rebound/blob/master/ipython_examples/SimulationArchive.ipynb)):

In [2]:
sim.initSimulationArchive("archive.bin", interval=1e3)
sim.integrate(1.e6)

Once we're ready to inspect our simulation, we just have to make sure to pass the `REBOUNDx` binary when creating the SimulationArchive to make sure the right effects and particle parameters are loaded:

In [3]:
sa = rebound.SimulationArchive("archive.bin", rebxfilename = "rebxarchive.bin")

We now create a different simulation from a snapshot in the `SimulationArchive` halfway through:

In [4]:
sim2 = sa[500]
sim2.t

500000.311856871

We now integrate our loaded simulation to the same time as above (1.e6):

In [5]:
sim2.integrate(1.e6)

and see that we obtain exactly the same particle positions in the original and reloaded simulations:

In [6]:
sim.status()

---------------------------------
REBOUND version:     	3.1.1
REBOUND built on:    	Jan  4 2017 09:03:10
Number of particles: 	3
Selected integrator: 	whfast
Simulation time:     	1000000.000000
Current timestep:    	0.314002
---------------------------------
<rebound.Particle object, m=1.0 x=-0.0014987956816867735 y=-0.0014917356304337854 z=0.0 vx=0.00070391125225057 vy=-0.0011135635648115408 vz=0.0>
<rebound.Particle object, m=0.001 x=1.105306583710834 y=-0.1700487535915524 z=0.0 vx=0.06434659981228147 vy=0.8839299450833844 vz=0.0>
<rebound.Particle object, m=0.001 x=0.3934890986071336 y=1.6617843854899825 z=0.0 vx=-0.7682578520628421 vy=0.22963361972814875 vz=0.0>
---------------------------------


In [7]:
sim2.status()

---------------------------------
REBOUND version:     	3.1.1
REBOUND built on:    	Jan  4 2017 09:03:10
Number of particles: 	3
Selected integrator: 	whfast
Simulation time:     	1000000.000000
Current timestep:    	0.314002
---------------------------------
<rebound.Particle object, m=1.0 x=-0.0014987956816867735 y=-0.0014917356304337854 z=0.0 vx=0.00070391125225057 vy=-0.0011135635648115408 vz=0.0>
<rebound.Particle object, m=0.001 x=1.105306583710834 y=-0.1700487535915524 z=0.0 vx=0.06434659981228147 vy=0.8839299450833844 vz=0.0>
<rebound.Particle object, m=0.001 x=0.3934890986071336 y=1.6617843854899825 z=0.0 vx=-0.7682578520628421 vy=0.22963361972814875 vz=0.0>
---------------------------------
